In [2]:
pip install torch 

   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.7 MB 667.8 kB/s eta 0:04:00
   ---------------------------------------- 0.1/159.7 MB 1.1 MB/s eta 0:02:32
   ---------------------------------------- 0.1/159.7 MB 950.9 kB/s eta 0:02:48
   ---------------------------------------- 0.2/159.7 MB 1.1 MB/s eta 0:02:32
   ---------------------------------------- 0.2/159.7 MB 1.0 MB/s eta 0:02:35
   ---------------------------------------- 0.3/159.7 MB 1.1 MB/s eta 0:02:23
   ---------------------------------------- 0.4/159.7 MB 1.1 MB/s eta 0:02:26
   ---------------------------------------- 0.4/159.7 MB 1.2 MB/s eta 0:02:13
   ---------------------------------------- 0.5/159.7 MB 1.2 MB/s eta 0:02:18
   ---------------------------------------- 0.6/159.7 MB 1.2 MB/s eta 0:02:11
   ---------------------------------------- 0.6/159.7 MB 1.2 MB/s eta 0:02:08
   ---------------------------------------- 0.7/159.7 MB 1.3 MB/s e

In [3]:
import numpy as np
import torch
import torch.nn as nn

In [4]:
#without using pytorch

In [10]:
#Defining the softmax layer

def softmax_layer(z):
    z= np.exp(z-z.max(axis=1, keepdims=True))
    return z/z.sum(axis=-1, keepdims=True)  

In [11]:
#Defining the self attention fucntion

def self_attention(X, mask, W_kqv, W_out):
    K,Q,V = np.split(X@W_kqv, 3, axis=1)
    attention = softmax_layer(K@Q.T / np.sqrt(X.shape[1]) +mask)
    return attention@V@W_out, attention

In [18]:
#Using pytorch library calculating the weights and output

T, d =100,64
attention = nn.MultiheadAttention(d,1,bias=False,batch_first=True)
M = torch.triu(-float('inf')*torch.ones(T,T),1)
X = torch.randn(1,T,d)
Y_,A_ = attention(X,X,X,attn_mask=M)

In [19]:
#Converts the tensor into a numpy array and transposes it

Y,A = self_attention(X[0].numpy(), M.numpy(),
                     attention.in_proj_weight.detach().numpy().T,
                     attention.out_proj.weight.detach().numpy().T)

In [22]:
np.linalg.norm(Y-Y_[0].detach().numpy())

1.6297648e-06

In [23]:
np.linalg.norm(A-A_[0].detach().numpy())

2.586149e-07

In [24]:
def self_attention_b(X, mask, W_kqv, W_out):
    K,Q,V = np.split(X@W_kqv, 3, axis=-1)
    attention = softmax_layer(K@Q.swapaxes(-1,-2) / np.sqrt(X.shape[-1]) +mask)
    return attention@V@W_out, attention

In [25]:
B,T,d = 20,100,64
X= torch.randn(B,T,d)

In [26]:
M = torch.triu(-float('inf')*torch.ones(T,T),1)

In [31]:
Y_,A_ = attention(X,X,X,attn_mask=M)

In [37]:
Y,A = self_attention_b(X[0].numpy(), M.numpy(),
                     attention.in_proj_weight.detach().numpy().T,
                     attention.out_proj.weight.detach().numpy().T)

In [38]:
np.linalg.norm(Y-Y_[0].detach().numpy())

1.5759381e-06

In [39]:
np.linalg.norm(A-A_[0].detach().numpy())

2.594832e-07

In [54]:
#MultiHead Attention